In [1]:
# import libraries 
import pandas as pd
import numpy as np
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import time 
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE



from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.svm import SVC
import sklearn.metrics
    
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection


/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth = 1000


In [3]:
df = pd.read_csv('../../data/processed/dortmund_trips.csv', index_col=0)

In [4]:
df

,datetime_start,b_number,latitude_start,p_name_start,longitude_start,datetime_end,latitude_end,p_name_end,longitude_end,trip_duration,...,hour,minute,day_of_year,towardsUniversity,awayFromUniversity,tripLabel,temperature °C,precipitation in mm,precipitation,area_start
0,2019-01-20 16:22:00,50641,51.506312,Hainallee / Südbad,7.470531,2019-01-20 17:00:00,51.493966,TU Dortmund Emil-Figge-Straße 50,7.418008,38,...,16,22,20,1,0,towardsUniversity,0.5,0.0,0,1
1,2019-01-20 16:42:00,53940,51.507457,Möllerbrücke,7.451364,2019-01-20 16:44:00,51.507457,Möllerbrücke,7.451364,2,...,16,42,20,0,0,noUniversityRide,0.5,0.0,0,1
2,2019-01-20 16:53:00,50061,51.503293,Vinckeplatz,7.455822,2019-01-20 17:13:00,51.519332,Cinestar,7.460124,20,...,16,53,20,0,0,noUniversityRide,0.5,0.0,0,1
3,2019-01-20 16:35:00,51138,51.499039,Steigenberger Hotel / Berswordtstr.,7.451472,2019-01-20 16:37:00,51.499039,Steigenberger Hotel / Berswordtstr.,7.451472,2,...,16,35,20,0,0,noUniversityRide,0.5,0.0,0,1
4,2019-01-20 16:43:00,53120,51.507457,Möllerbrücke,7.451364,2019-01-20 17:02:00,51.512836,Am Kaiserbrunnen,7.482258,19,...,16,43,20,0,0,noUniversityRide,0.5,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207471,2019-12-31 09:10:00,500084,51.532685,Immermannstr./Klinikzentrum,7.454631,2019-12-31 09:21:00,51.516831,Kuckelke,7.469189,11,...,9,10,365,0,0,noUniversityRide,3.2,0.0,0,3
207472,2019-12-31 09:07:00,500011,51.532685,Immermannstr./Klinikzentrum,7.454631,2019-12-31 09:21:00,51.516831,Kuckelke,7.469189,14,...,9,7,365,0,0,noUniversityRide,3.2,0.0,0,3
207473,2019-12-31 09:11:00,500837,51.492690,Universität/S-Bahnhof,7.417633,2019-12-31 09:31:00,51.512609,Hansastr.,7.463483,20,...,9,11,365,0,1,awayFromUniversity,3.2,0.0,0,2
207474,2019-12-31 09:57:00,500118,51.518297,Geschwister-Scholl-Str.,7.474962,2019-12-31 10:08:00,51.512609,Hansastr.,7.463483,11,...,9,57,365,0,0,noUniversityRide,3.2,0.0,0,3


In [6]:
y = df['tripLabel']
# use only start-information to classify the trip-class
X = df[['weekend', 'hour', 'distanceToUniversity', 'month', 'area_start']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [7]:
Algorithm = []
Precision = []
Recall = []
F1score = []
Support = []
exetime = [] 
desc = []

## LogisticRegression

### Binary Logistic Regression

In [8]:
y_away = df['awayFromUniversity']
y_towards = df['towardsUniversity']

In [9]:
X_train, X_test, y_away_train, y_away_test = train_test_split(X,y_away,test_size=0.3)

In [10]:
# Binary Logistic Regression (awayFromUniversity) 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = linear_model.LogisticRegression()
mod.fit(X_train_scaled,y_away_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_away_test,y_pred,average='weighted')
print(classification_report(y_away_test, y_pred))

Algorithm.append("Binary Logistic Regression")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts awayFromUniversity (is complementary to 2nd model)")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56972
           1       0.99      1.00      1.00      5271

    accuracy                           1.00     62243
   macro avg       1.00      1.00      1.00     62243
weighted avg       1.00      1.00      1.00     62243



In [11]:
X_train, X_test, y_towards_train, y_towards_test = train_test_split(X,y_towards,test_size=0.3)

In [12]:
# Binary Logistic Regression (towardsUniversity) 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = linear_model.LogisticRegression()
mod.fit(X_train_scaled,y_towards_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_towards_test,y_pred,average='weighted')
print(classification_report(y_towards_test, y_pred))

Algorithm.append("Binary Logistic Regression")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts towardsUniversity (is complementary to 1st model)")

              precision    recall  f1-score   support

           0       0.92      0.99      0.96     57120
           1       0.53      0.07      0.12      5123

    accuracy                           0.92     62243
   macro avg       0.73      0.53      0.54     62243
weighted avg       0.89      0.92      0.89     62243



### Multilinear Logistic Regression

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [14]:
# Multilinear Logistic Regression
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = linear_model.LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
mod.fit(X_train_scaled,y_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_test,y_pred,average='weighted')
print(classification_report(y_test, y_pred))

Algorithm.append("Multinomial Logistic Regression")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts tripLabel")

                    precision    recall  f1-score   support

awayFromUniversity       0.63      0.86      0.73      3072
  noUniversityRide       0.96      0.98      0.97     54007
 towardsUniversity       0.47      0.22      0.30      5164

          accuracy                           0.91     62243
         macro avg       0.68      0.69      0.67     62243
      weighted avg       0.90      0.91      0.90     62243



F1 is usually more useful than accuracy, especially if you have an uneven class distribution.

## Optimize the Hyperparameters for (binary) LogisticRegression

In [15]:
# How to optimize hyper-parameters of a binary Logistic Regression model using Grid Search
def optimize_hyperparameters(X,y):

    # Create an scaler object
    sc = StandardScaler()

    # Create a pca object
    pca = decomposition.PCA()

    # Create a logistic regression object with an L2 penalty
    logistic = linear_model.LogisticRegression()

    # Create a pipeline of three steps. First, standardize the data.
    # Second, tranform the data with PCA.
    # Third, train a logistic regression on the data.
    pipe = Pipeline(steps=[('sc', sc),
                           ('pca', pca),
                           ('logistic', logistic)])

    # Create Parameter Space
    # Create a list of a sequence of integers from 1 to 30 (the number of features in X + 1)
    n_components = list(range(1,X.shape[1]+1,1))
    # Create a list of values of the regularization parameter
    C = np.logspace(-4, 4, 50)
    # Create a list of options for the regularization penalty
    penalty = ['l1', 'l2']
    # Create a dictionary of all the parameter options 
    # Note has you can access the parameters of steps of a pipeline by using '__’
    parameters = dict(pca__n_components=n_components,
                      logistic__C=C,
                      logistic__penalty=penalty)

    # Conduct Parameter Optmization With Pipeline
    # Create a grid search object
    clf = GridSearchCV(pipe, parameters)

    # Fit the grid search
    clf.fit(X, y)
    # View The Best Parameters
    print('Best Penalty:', clf.best_estimator_.get_params()['logistic__penalty'])
    print('Best C:', clf.best_estimator_.get_params()['logistic__C'])
    print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])
    print(); print(clf.best_estimator_.get_params()['logistic'])

    # Use Cross Validation To Evaluate Model
    CV_Result = cross_val_score(clf, X, y, cv=4, n_jobs=-1)
    print(); print(CV_Result)
    print(); print(CV_Result.mean())
    print(); print(CV_Result.std())
    


In [16]:
y_towards = df['towardsUniversity']
y_away = df['awayFromUniversity']

# use only start-information to classify the trip-class
X = df[['weekend', 'hour', 'distanceToUniversity', 'month', 'area_start']]

### These are the best parameters for logistic regression for the attribute awayFromUniversity

In [18]:
optimize_hyperparameters(X,y_away)

Best Penalty: l2
Best C: 0.12648552168552957
Best Number Of Components: 4

LogisticRegression(C=0.12648552168552957, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

[0.9854441  0.98681293 0.98557905 0.98640807]

0.9860610383851627

0.0005698014886013382


### These are the best parameters for logistic regression for the attribute towardsUniversity

In [19]:
optimize_hyperparameters(X,y_towards)

Best Penalty: l2
Best C: 0.8286427728546842
Best Number Of Components: 5

LogisticRegression(C=0.8286427728546842, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

[0.91808209 0.91808209 0.91684821 0.91573001]

0.9171856021901329

0.0009797906933469204


### Performance of the models

In [17]:
pca_model = PCA(n_components=4).fit(X)
X_pca = pca_model.transform(X)

In [18]:
X_train, X_test, y_away_train, y_away_test = train_test_split(X_pca, y_away, test_size=0.3)

In [19]:
# Binary Logistic Regression (awayFromUniversity) 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = linear_model.LogisticRegression(C=0.12648552168552957, class_weight=None, dual=False,
                                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                                   warm_start=False)

mod.fit(X_train_scaled,y_away_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_away_test,y_pred,average='weighted')
print(classification_report(y_away_test, y_pred))

Algorithm.append("Binary Logistic Regression")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Optimized hyperparameters of model in index 0")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     57052
           1       0.96      1.00      0.98      5191

    accuracy                           1.00     62243
   macro avg       0.98      1.00      0.99     62243
weighted avg       1.00      1.00      1.00     62243



In [20]:
X_train, X_test, y_towards_train, y_towards_test = train_test_split(X,y_towards,test_size=0.3)

In [21]:
# Binary Logistic Regression (towardsUniversity) 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = linear_model.LogisticRegression(C=0.8286427728546842, class_weight=None, dual=False,
                                       fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                                       max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                                       random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                                       warm_start=False)
mod.fit(X_train_scaled,y_towards_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_towards_test,y_pred,average='weighted')
print(classification_report(y_towards_test, y_pred))

Algorithm.append("Binary Logistic Regression")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Optimized hyperparameters of model in index 1")

              precision    recall  f1-score   support

           0       0.92      0.99      0.96     57170
           1       0.53      0.07      0.12      5073

    accuracy                           0.92     62243
   macro avg       0.73      0.53      0.54     62243
weighted avg       0.89      0.92      0.89     62243



In [22]:
result_dict = {"Algorithm":Algorithm,
              "Precision":Precision, 
              "Recall": Recall,
              "F1score": F1score,
              "Execution time (sec)":exetime,
              "Description":desc}

df_result = pd.DataFrame(result_dict)
df_result

,Algorithm,Precision,Recall,F1score,Execution time (sec),Description
0,Binary Logistic Regression,0.999299,0.999293,0.999294,0.752478,Predicts awayFromUniversity (is complementary to 2nd model)
1,Binary Logistic Regression,0.890065,0.918320,0.887933,0.250331,Predicts towardsUniversity (is complementary to 1st model)
2,Multinomial Logistic Regression,0.898683,0.913886,0.901719,8.989423,Predicts tripLabel
3,Binary Logistic Regression,0.996802,0.996674,0.996704,0.537564,Optimized hyperparameters of model in index 0
4,Binary Logistic Regression,0.891143,0.919107,0.889501,0.259306,Optimized hyperparameters of model in index 1


## SVM

In [23]:
Algorithm = []
Precision = []
Recall = []
F1score = []
Support = []
exetime = [] 
desc = []

### linear

In [24]:
# SVM for awayFromUniversity
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = SVC(kernel='linear')
mod.fit(X_train_scaled,y_away_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_away_test,y_pred,average='weighted')
print(classification_report(y_away_test, y_pred))

Algorithm.append("SVM with linear kernel")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts awayFromUniversity (is complementary to 2nd model)")

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     57052
           1       0.00      0.00      0.00      5191

    accuracy                           0.92     62243
   macro avg       0.46      0.50      0.48     62243
weighted avg       0.84      0.92      0.88     62243



In [25]:
# SVM for towardsUniversity
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = SVC(kernel='linear')
mod.fit(X_train_scaled,y_towards_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_towards_test,y_pred,average='weighted')
print(classification_report(y_towards_test, y_pred))

Algorithm.append("SVM with linear kernel")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts towardsUniversity (is complementary to 1st model)")

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     57170
           1       0.00      0.00      0.00      5073

    accuracy                           0.92     62243
   macro avg       0.46      0.50      0.48     62243
weighted avg       0.84      0.92      0.88     62243



### rbf

In [26]:
# SVM for awayFromUniversity
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = SVC(kernel='rbf')
mod.fit(X_train_scaled,y_away_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_away_test,y_pred,average='weighted')
print(classification_report(y_away_test, y_pred))

Algorithm.append("SVM with rbf kernel")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts awayFromUniversity (is complementary to 4th model)")

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     57052
           1       0.00      0.00      0.00      5191

    accuracy                           0.92     62243
   macro avg       0.46      0.50      0.48     62243
weighted avg       0.84      0.92      0.88     62243



In [27]:
# SVM for towardsUniversity 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

start = time.time()
mod = SVC(kernel='rbf')
mod.fit(X_train_scaled,y_towards_train)
y_pred = mod.predict(X_test_scaled)
end = time.time()

precision,recall,fscore,support=score(y_towards_test,y_pred,average='weighted')
print(classification_report(y_towards_test, y_pred))

Algorithm.append("SVM with rbf kernel")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts towardsUniversity (is complementary to 3rd model)")

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     57170
           1       0.72      0.05      0.10      5073

    accuracy                           0.92     62243
   macro avg       0.82      0.53      0.53     62243
weighted avg       0.91      0.92      0.89     62243



In [28]:
def optimize_hyperparameters_svm(X,y, nfolds):
    # defining parameter range 
    param_grid = [{'C': [0.1, 1, 10, 300], 'kernel': ['linear']},
                  {'C': [0.1, 1, 10, 300],  
                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                  'kernel': ['rbf']}]

    grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv=nfolds) 

    # fitting the model for grid search 
    grid.fit(X, y) 
    
    # print best parameter after tuning 
    print(grid.best_params_) 
  
    # print how our model looks after hyper-parameter tuning 
    print(grid.best_estimator_)

In [29]:
result_dict = {"Algorithm":Algorithm,
              "Precision":Precision, 
              "Recall": Recall,
              "F1score": F1score,
              "Execution time (sec)":exetime,
              "Description":desc}

df_result = pd.DataFrame(result_dict)
df_result

,Algorithm,Precision,Recall,F1score,Execution time (sec),Description
0,SVM with linear kernel,0.840158,0.916601,0.876716,105.369310,Predicts awayFromUniversity (is complementary to 2nd model)
1,SVM with linear kernel,0.843636,0.918497,0.879477,97.975079,Predicts towardsUniversity (is complementary to 1st model)
2,SVM with rbf kernel,0.840158,0.916601,0.876716,297.771291,Predicts awayFromUniversity (is complementary to 4th model)
3,SVM with rbf kernel,0.905629,0.921148,0.888784,592.105684,Predicts towardsUniversity (is complementary to 3rd model)


#### hyperparameters cannot be optimized (due to computation power)

## Random Forrest to predict tripLabel

In [30]:
Algorithm = []
Precision = []
Recall = []
F1score = []
Support = []
exetime = [] 
desc = []

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [32]:
start = time.time()
mod = RandomForestClassifier()
mod.fit(X_train,y_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

precision,recall,fscore,support=score(y_test,y_pred,average='weighted')

Algorithm.append("Random Forrest")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts tripLabel")

=== Confusion Matrix ===
[[ 2528     0   614]
 [    0 53273   691]
 [ 1048  2244  1845]]


=== Classification Report ===
                    precision    recall  f1-score   support

awayFromUniversity       0.71      0.80      0.75      3142
  noUniversityRide       0.96      0.99      0.97     53964
 towardsUniversity       0.59      0.36      0.45      5137

          accuracy                           0.93     62243
         macro avg       0.75      0.72      0.72     62243
      weighted avg       0.92      0.93      0.92     62243



=== All AUC Scores ===
[0.33622518 0.87478311 0.87078273 0.86813187 0.87724118 0.8954598
 0.8893334  0.87169229 0.78367957 0.91357787]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8180906992631833


### Optimize Parameters of RandomForrest

In [33]:
def optimize_hyperparameters_randomforest(X,y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # random forest model creation
    rfc = RandomForestClassifier()
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    # Random search of parameters
    rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the model
    rfc_random.fit(X_train, y_train)
    # print results
    print(rfc_random.best_params_)

In [11]:
optimize_hyperparameters_randomforest(X,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 36.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 174.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 339.9min finished


{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}


### Performance

In [34]:
start = time.time()
mod = RandomForestClassifier(n_estimators=1000, min_samples_split = 2,min_samples_leaf=1, max_depth=10, max_features='auto', bootstrap= False)
mod.fit(X_train,y_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

precision,recall,fscore,support=score(y_test,y_pred,average='weighted')

Algorithm.append("Random Forrest")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Optimized hyperparameters of model in index 0")

=== Confusion Matrix ===
[[ 2693     0   449]
 [    0 53815   149]
 [ 1140  2678  1319]]


=== Classification Report ===
                    precision    recall  f1-score   support

awayFromUniversity       0.70      0.86      0.77      3142
  noUniversityRide       0.95      1.00      0.97     53964
 towardsUniversity       0.69      0.26      0.37      5137

          accuracy                           0.93     62243
         macro avg       0.78      0.70      0.71     62243
      weighted avg       0.92      0.93      0.91     62243



=== All AUC Scores ===
[0.35420281 0.88133796 0.88220551 0.87651822 0.88018122 0.91493156
 0.91401166 0.91025208 0.89709356 0.92263942]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8433374011794988


## Binary RandomForest
### Prediction model for both attributes separately


#### AwayFromUniversity

In [35]:
X_train, X_test, y_away_train, y_away_test = train_test_split(X,y_away,test_size=0.3)

In [36]:
start = time.time()
mod = RandomForestClassifier()
mod.fit(X_train,y_away_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y_away, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_away_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_away_test, y_pred))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

precision,recall,fscore,support=score(y_away_test,y_pred,average='weighted')

Algorithm.append("Random Forrest")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts awayFromUniversity (complement)")

=== Confusion Matrix ===
[[56886     0]
 [    0  5357]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56886
           1       1.00      1.00      1.00      5357

    accuracy                           1.00     62243
   macro avg       1.00      1.00      1.00     62243
weighted avg       1.00      1.00      1.00     62243



=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  1.0


In [13]:
optimize_hyperparameters_randomforest(X,y_away)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 303.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 340.8min finished


{'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}


In [37]:
start = time.time()
mod = RandomForestClassifier(n_estimators=400, min_samples_split = 5,min_samples_leaf=1, max_depth=30, max_features='sqrt', bootstrap= True)
mod.fit(X_train,y_away_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y_away, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_away_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_away_test, y_pred))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

precision,recall,fscore,support=score(y_away_test,y_pred,average='weighted')

Algorithm.append("Random Forrest")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Optimized hyperparameters of model in index 2")


=== Confusion Matrix ===
[[56886     0]
 [    0  5357]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56886
           1       1.00      1.00      1.00      5357

    accuracy                           1.00     62243
   macro avg       1.00      1.00      1.00     62243
weighted avg       1.00      1.00      1.00     62243



=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  1.0


### towardsUniversity

In [38]:
X_train, X_test, y_towards_train, y_towards_test = train_test_split(X,y_towards,test_size=0.3)

In [39]:
start = time.time()
mod = RandomForestClassifier()
mod.fit(X_train,y_towards_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y_away, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_towards_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_towards_test, y_pred))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

precision,recall,fscore,support=score(y_towards_test,y_pred,average='weighted')

Algorithm.append("Random Forrest")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Predicts towardsUniversity (complement)")

=== Confusion Matrix ===
[[55775  1426]
 [ 3221  1821]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.95      0.98      0.96     57201
           1       0.56      0.36      0.44      5042

    accuracy                           0.93     62243
   macro avg       0.75      0.67      0.70     62243
weighted avg       0.91      0.93      0.92     62243



=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  1.0


In [15]:
optimize_hyperparameters_randomforest(X,y_towards)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 32.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 152.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 293.0min finished


{'n_estimators': 2000, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}


In [40]:
start = time.time()
mod = RandomForestClassifier(n_estimators=2000, min_samples_split = 5,min_samples_leaf=1, max_depth=10, max_features='sqrt', bootstrap= True)
mod.fit(X_train,y_towards_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y_away, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_towards_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_towards_test, y_pred))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

precision,recall,fscore,support=score(y_towards_test,y_pred,average='weighted')

Algorithm.append("Random Forrest")
Precision.append(precision)
Recall.append(recall)
F1score.append(fscore)
Support.append(support)
exetime.append((end-start))
desc.append("Optimized hyperparameters of model in index 4")

=== Confusion Matrix ===
[[56609   592]
 [ 3836  1206]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.94      0.99      0.96     57201
           1       0.67      0.24      0.35      5042

    accuracy                           0.93     62243
   macro avg       0.80      0.61      0.66     62243
weighted avg       0.92      0.93      0.91     62243



=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  1.0


In [41]:
result_dict = {"Algorithm":Algorithm,
              "Precision":Precision, 
              "Recall": Recall,
              "F1score": F1score,
              "Execution time (sec)":exetime,
              "Description":desc}

df_result = pd.DataFrame(result_dict)
df_result

,Algorithm,Precision,Recall,F1score,Execution time (sec),Description
0,Random Forrest,0.915971,0.926144,0.918487,16.526235,Predicts tripLabel
1,Random Forrest,0.918142,0.929052,0.914644,131.005115,Optimized hyperparameters of model in index 0
2,Random Forrest,1.000000,1.000000,1.000000,5.359210,Predicts awayFromUniversity (complement)
3,Random Forrest,1.000000,1.000000,1.000000,18.891498,Optimized hyperparameters of model in index 2
4,Random Forrest,0.914250,0.925341,0.917834,13.856886,Predicts towardsUniversity (complement)
5,Random Forrest,0.915007,0.928859,0.912970,239.189682,Optimized hyperparameters of model in index 4


# Performance of finally chosen model

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [43]:
start = time.time()
mod = RandomForestClassifier(n_estimators=1000, min_samples_split = 2,min_samples_leaf=1, max_depth=10, max_features='auto', bootstrap= False)
mod.fit(X_train,y_train)
y_pred = mod.predict(X_test)
end = time.time()

rfc_cv_score = cross_val_score(mod, X, y, cv=10)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred))
print('\n')

print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())




=== Confusion Matrix ===
[[ 2603     0   470]
 [    0 53986   111]
 [ 1118  2685  1270]]


=== Classification Report ===
                    precision    recall  f1-score   support

awayFromUniversity       0.70      0.85      0.77      3073
  noUniversityRide       0.95      1.00      0.97     54097
 towardsUniversity       0.69      0.25      0.37      5073

          accuracy                           0.93     62243
         macro avg       0.78      0.70      0.70     62243
      weighted avg       0.92      0.93      0.91     62243



=== All AUC Scores ===
[0.3582032  0.88133796 0.8823983  0.87651822 0.88018122 0.91507615
 0.91396346 0.90948089 0.89781655 0.92263942]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8437615379811207


In [44]:
# 10 fold cross validation
print (np.mean(cross_val_score(mod, X_train, y_train, cv=10)))

0.9280604153986042
